# Delta calculator

A simple delta calculator

In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes.greeks.numerical as greeks
import py_vollib.black_scholes.implied_volatility as iv
import py_vollib.black_scholes as bs
from typing import Tuple

In [2]:
# parameters
spot:float = 17000
strike:float = 17000
risk_free_rate:float = 0.1
iv_range: Tuple[float, float] = (0.1,0.3)
step = 100 
de = 1 # days to expiry


In [3]:
kwargs = {
    'flag': 'c',
    'S': spot,
    'K': strike,
    't': de/360,
    'r': risk_free_rate
    
}

In [4]:
ivs = np.linspace(iv_range[0], iv_range[-1], 10)
strikes = set() 
for i in range(5):
    strikes.add(spot + i*step)
    strikes.add(spot - i*step)
    


In [5]:
calls = []
puts = []
for sigma in ivs:
    for strk in strikes:
        kwargs.update({'sigma': sigma, 'K': strk, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, strk, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, strk, price, grk))
        

## Call option Delta (by strike)

**Call option delta for a fixed spot price with various strikes and implied volatility**

In [6]:
df1 = pd.DataFrame(calls, columns=['iv', 'strike', 'price', 'greek']).round(4)
df1_pivot = df1.pivot(index='iv', columns='strike', values='greek').round(4)

## Put option Delta (by strike)
**Put option delta for a fixed spot price with various strikes and implied volatility**

In [7]:
df2 = pd.DataFrame(puts, columns=['iv', 'strike', 'price', 'greek']).round(4)
df2_pivot = df2.pivot(index='iv', columns='strike', values='greek').round(4)

# Combined Delta - fixed spot

In [8]:
pvt = (df1_pivot + df2_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

strike,16600,16700,16800,16900,17000,17100,17200,17300,17400
iv,,,,,,,,,
0.1,1.000000,0.999400,0.978600,0.759800,0.044200,-0.709800,-0.969600,-0.999000,-1.000000
0.1222,0.999800,0.995000,0.940400,0.664000,0.037000,-0.612400,-0.923200,-0.992400,-0.999600
0.1444,0.998400,0.982600,0.889200,0.585000,0.032200,-0.534600,-0.865400,-0.976000,-0.997400
0.1667,0.994000,0.960800,0.833400,0.520800,0.028800,-0.472400,-0.804800,-0.949400,-0.991000
0.1889,0.984600,0.931400,0.778200,0.468400,0.026200,-0.422000,-0.746600,-0.915400,-0.978800
0.2111,0.970000,0.897000,0.726200,0.425000,0.024400,-0.380600,-0.692600,-0.876800,-0.960600
0.2333,0.950600,0.860200,0.678400,0.388800,0.023000,-0.346000,-0.643600,-0.836600,-0.937400
0.2556,0.927400,0.822800,0.635000,0.358400,0.021800,-0.316800,-0.599600,-0.796400,-0.910800
0.2778,0.901600,0.786000,0.596000,0.332200,0.021000,-0.291800,-0.560200,-0.757400,-0.881800


In [9]:
calls = []
puts = []
spot_vals = np.linspace(spot*0.98,spot*1.02,10)
for sigma in ivs:
    for spt in spot_vals:
        kwargs.update({'sigma': sigma, 'S': spt, 'K':strike, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, spt, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, spt, price, grk))

## Call option delta (by spot)

**Call option delta for a fixed strike price with various spot prices and implied volatility**


In [10]:
df3 = pd.DataFrame(calls, columns=['iv', 'spot', 'price', 'greek']).round(4)
df3['spot'] = df3.spot.astype(int)
df3_pivot = df3.pivot(index='iv', columns='spot', values='greek').round(4)
df3_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1000,0.0001,0.0018,0.0195,0.1124,0.3569,0.6831,0.9059,0.9843,0.9986,0.9999
0.1222,0.0010,0.0085,0.0457,0.1606,0.3825,0.6521,0.8594,0.9609,0.9927,0.9991
0.1444,0.0045,0.0217,0.0767,0.2009,0.4005,0.6300,0.8194,0.9321,0.9807,0.9959
0.1667,0.0118,0.0402,0.1082,0.2341,0.4140,0.6136,0.7859,0.9022,0.9635,0.9890
0.1889,0.0229,0.0615,0.1380,0.2614,0.4244,0.6010,0.7581,0.8734,0.9433,0.9784
0.2111,0.0371,0.0840,0.1651,0.2841,0.4327,0.5910,0.7349,0.8470,0.9219,0.9649
0.2333,0.0533,0.1064,0.1895,0.3032,0.4395,0.5829,0.7153,0.8231,0.9004,0.9494
0.2556,0.0704,0.1278,0.2112,0.3194,0.4452,0.5762,0.6987,0.8017,0.8797,0.9329
0.2778,0.0879,0.1481,0.2306,0.3334,0.4500,0.5706,0.6844,0.7826,0.8601,0.9160


## Put option delta (by spot)

**Put option delta for a fixed strike price with various spot prices and implied volatility**


In [11]:
df4 = pd.DataFrame(puts, columns=['iv', 'spot', 'price', 'greek']).round(4)
df4['spot'] = df4.spot.astype(int)
df4_pivot = df4.pivot(index='iv', columns='spot', values='greek').round(4)
df4_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1000,-0.9999,-0.9982,-0.9805,-0.8876,-0.6431,-0.3169,-0.0941,-0.0157,-0.0014,-0.0001
0.1222,-0.9990,-0.9915,-0.9543,-0.8394,-0.6175,-0.3479,-0.1406,-0.0391,-0.0073,-0.0009
0.1444,-0.9955,-0.9783,-0.9233,-0.7991,-0.5995,-0.3700,-0.1806,-0.0679,-0.0193,-0.0041
0.1667,-0.9882,-0.9598,-0.8918,-0.7659,-0.5860,-0.3864,-0.2141,-0.0978,-0.0365,-0.0110
0.1889,-0.9771,-0.9385,-0.8620,-0.7386,-0.5756,-0.3990,-0.2419,-0.1266,-0.0567,-0.0216
0.2111,-0.9629,-0.9160,-0.8349,-0.7159,-0.5673,-0.4090,-0.2651,-0.1530,-0.0781,-0.0351
0.2333,-0.9467,-0.8936,-0.8105,-0.6968,-0.5605,-0.4171,-0.2847,-0.1769,-0.0996,-0.0506
0.2556,-0.9296,-0.8722,-0.7888,-0.6806,-0.5548,-0.4238,-0.3013,-0.1983,-0.1203,-0.0671
0.2778,-0.9121,-0.8519,-0.7694,-0.6666,-0.5500,-0.4294,-0.3156,-0.2174,-0.1399,-0.0840


# Combined Delta - fixed strike

Combined delta for a specific strike but with different volatility and spot prices

In [12]:
pvt = (df3_pivot + df4_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1,-0.999800,-0.996400,-0.961000,-0.775200,-0.286200,0.366200,0.811800,0.968600,0.997200,0.999800
0.1222,-0.998000,-0.983000,-0.908600,-0.678800,-0.235000,0.304200,0.718800,0.921800,0.985400,0.998200
0.1444,-0.991000,-0.956600,-0.846600,-0.598200,-0.199000,0.260000,0.638800,0.864200,0.961400,0.991800
0.1667,-0.976400,-0.919600,-0.783600,-0.531800,-0.172000,0.227200,0.571800,0.804400,0.927000,0.978000
0.1889,-0.954200,-0.877000,-0.724000,-0.477200,-0.151200,0.202000,0.516200,0.746800,0.886600,0.956800
0.2111,-0.925800,-0.832000,-0.669800,-0.431800,-0.134600,0.182000,0.469800,0.694000,0.843800,0.929800
0.2333,-0.893400,-0.787200,-0.621000,-0.393600,-0.121000,0.165800,0.430600,0.646200,0.800800,0.898800
0.2556,-0.859200,-0.744400,-0.577600,-0.361200,-0.109600,0.152400,0.397400,0.603400,0.759400,0.865800
0.2778,-0.824200,-0.703800,-0.538800,-0.333200,-0.100000,0.141200,0.368800,0.565200,0.720200,0.832000
